# ARC CHALLENGE AFRICA SOLUTION WALKTHROUGH

This notebook details the approach and implementation for solving tasks in the Abstraction and Reasoning Challenge (ARC). The goal is to develop a system that can infer the underlying rules from a few input-output examples and apply them to new test cases.

My solution is a logic-based pipeline that attempts to identify patterns and transformations within the provided training examples.

## Setup and Data Loading

This section sets up the necessary environment by installing required libraries and loading the training and testing data. This corresponds to **Step 1: Getting Familiar with the Task Format** from the walkthrough document.

In [1]:
# Install required libraries for visualization and widgets
!pip install ipywidgets --quiet
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


**TRAIN INPUT AND OUTPUT VISUALIZATIONS**

In [2]:
# Reload extensions for ipywidgets
%reload_ext autoreload
%reload_ext widgetsnbextension

In [3]:
# Import necessary libraries
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings

# Ignore warnings for cleaner output during execution
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Load training data from the JSON file
# This is part of getting familiar with the task format as mentioned in the walkthrough.
with open('/content/train.json', 'r') as f:
    train_data = json.load(f)

# Define a colormap for visualizing the grids (numbers 0-9 represent colors)
cmap = plt.cm.get_cmap('tab20', 10)

# Function to display input and output grids side-by-side visually
# This is part of the interactive viewer described in Step 1.
def show_image_only(input_grid, output_grid, task_id=''):
    """
    Show image only function. Displays the input and output grids as images with a color key.
    """
    input_grid = np.array(input_grid)
    output_grid = np.array(output_grid)

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(f"{task_id} - Input vs Output", fontsize=14, weight='bold')

    ax1 = fig.add_subplot(1, 3, 1)
    ax1.imshow(input_grid, cmap=cmap, vmin=0, vmax=9)
    ax1.set_title("Input Grid")
    ax1.axis('off')

    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(output_grid, cmap=cmap, vmin=0, vmax=9)
    ax2.set_title("Output Grid")
    ax2.axis('off')

    ax3 = fig.add_subplot(1, 3, 3)
    legend = np.arange(10).reshape(-1, 1)
    ax3.imshow(legend, cmap=cmap, vmin=0, vmax=9)
    ax3.set_title("Color Key", fontsize=10)
    ax3.set_yticks(range(10))
    ax3.set_yticklabels(range(10))
    ax3.set_xticks([])
    ax3.grid(False)

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()

# Function to create ipywidgets Output widgets to display matrix representations of grids
# This is also part of the interactive viewer in Step 1, allowing inspection of raw data.
def make_matrix_widgets(input_grid, output_grid):
    input_matrix = widgets.Output()
    output_matrix = widgets.Output()

    with input_matrix:
        display(HTML("<pre style='font-family: monospace;'>" + str(np.array(input_grid)) + "</pre>"))

    with output_matrix:
        display(HTML("<pre style='font-family: monospace;'>" + str(np.array(output_grid)) + "</pre>"))

    return input_matrix, output_matrix

# Function to display a specific training task using the viewer components
# This combines the image and matrix visualizations for a single task.
def display_train_task(task_ids, index):
    task_id = task_ids[index]
    print(f"\n📦 Viewing Train Task: {task_id} ({index+1}/{len(task_ids)})")

    example = train_data[task_id]['train'][0]
    train_input = example['input']
    train_output = example['output']

    show_image_only(train_input, train_output, task_id=task_id)

    input_matrix, output_matrix = make_matrix_widgets(train_input, train_output)

    # Toggle buttons to show/hide the matrix representations
    toggle_in = widgets.ToggleButton(value=False, description="Show Input Matrix")
    toggle_out = widgets.ToggleButton(value=False, description="Show Output Matrix")

    # Arrange the toggle buttons and matrix outputs in a horizontal box
    box = widgets.HBox([
        widgets.VBox([toggle_in, input_matrix]),
        widgets.VBox([toggle_out, output_matrix])
    ])

    # Update function to show/hide matrices based on toggle button state
    def update_matrix(change):
        input_matrix.layout.display = 'block' if toggle_in.value else 'none'
        output_matrix.layout.display = 'block' if toggle_out.value else 'none'
        toggle_in.description = "Hide Input Matrix" if toggle_in.value else "Show Input Matrix"
        toggle_out.description = "Hide Output Matrix" if toggle_out.value else "Show Output Matrix"

    # Observe changes in toggle button values to update matrix display
    toggle_in.observe(update_matrix, names='value')
    toggle_out.observe(update_matrix, names='value')

    # Initial update to set correct visibility
    update_matrix(None)
    display(box)

# Setup navigation for iterating through training tasks
train_ids = list(train_data.keys())
train_index = 0
prev_button = widgets.Button(description="⏮️ Previous")
next_button = widgets.Button(description="▶️ Next")
jump_box = widgets.BoundedIntText(value=1, min=1, max=len(train_ids), description="Jump to:")
jump_button = widgets.Button(description="Go")
output_area = widgets.Output() # Output area to display the task viewer

# Navigation button handlers to update the displayed task
def render_current_task():
    with output_area:
        clear_output(wait=True)
        display_train_task(train_ids, train_index)

def on_prev_clicked(b):
    global train_index
    train_index = (train_index - 1) % len(train_ids)
    render_current_task()

def on_next_clicked(b):
    global train_index
    train_index = (train_index + 1) % len(train_ids)
    render_current_task()

def on_jump_clicked(b):
    global train_index
    new_index = jump_box.value - 1
    if 0 <= new_index < len(train_ids):
        train_index = new_index
        render_current_task()

# Link button clicks to their respective handler functions
prev_button.on_click(on_prev_clicked)
next_button.on_click(on_next_clicked)
jump_button.on_click(on_jump_clicked)

# Display the title and navigation UI for the training task viewer
display(HTML("""
    <div style='width: 100%; text-align: 20%; padding-bottom: 10px;'>
        <h2 style='color:#1f77b4; font-weight:bold; font-size:24px;'>
            🚁 Train Inputs and Outputs
        </h2>
    </div>
"""))
nav_ui = widgets.HBox([prev_button, next_button, jump_box, jump_button])
display(nav_ui, output_area)

# Render the initial training task
render_current_task()

Output()

## My Prediction Model

This section contains the core logic of the hybrid solution, combining pattern recognition, geometric transformations, and fallback strategies to predict the output grids for the test tasks. This section covers **Steps 2 through 6** of the walkthrough document.

In [4]:
import json
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist
from skimage.transform import resize
import re
import itertools
from tqdm import tqdm

def arc_agi_hybrid_solution(
    train_json_path="/content/train.json",
    test_json_path="/content/test.json",
    sample_csv_path="/content/SampleSubmission.csv",
    output_csv_path="/content/arc_agi_final_submission.csv"):
    """
    Implements a hybrid solution for the Abstraction and Reasoning Challenge (ARC).
    This function loads training and testing data, analyzes training examples to infer
    patterns and transformations (color mappings, geometric, structural), applies these
    inferred rules or fallback strategies to test inputs, and saves the predicted output
    grids to a submission CSV file, ensuring the correct format and shape.

    Args:
        train_json_path (str): Path to the training JSON file.
        test_json_path (str): Path to the testing JSON file.
        sample_csv_path (str): Path to the sample submission CSV file.
        output_csv_path (str): Path where the final submission CSV will be saved.
    """
    # Load data for training and testing tasks
    with open(train_json_path, 'r') as f:
        train_tasks = json.load(f)
    with open(test_json_path, 'r') as f:
        test_tasks = json.load(f)

    # Detect delimiter from sample submission file to format output correctly
    # This corresponds to Step 2: Inferring the Output Grid Size and formatting.
    df_sample = pd.read_csv(sample_csv_path, dtype={'ID': str, 'row': str}) # Read 'row' column as string
    use_space = ' ' in df_sample['row'].iloc[0]
    join_delim = ' ' if use_space else ''

    # Build expected output shapes from the sample submission file
    # This is part of Step 2, determining the target dimensions.
    task_shapes = {}
    df_sample['task_id'] = df_sample['ID'].str.rsplit('_', n=1).str[0]
    for task_id, grp in df_sample.groupby('task_id'):
        row_sample = grp['row'].iloc[0]
        # Determine columns based on delimiter or string length
        ncols = len(row_sample.split(join_delim)) if use_space else len(row_sample)
        task_shapes[task_id] = (len(grp), ncols)

    # Define a library of potential transformations
    # This supports the various strategies described in Steps 3, 4, and 5.
    TRANSFORMATIONS = {
        'flip_horizontal': lambda g: np.flip(g, axis=1),
        'flip_vertical': lambda g: np.flip(g, axis=0),
        'rotate_90': lambda g: np.rot90(g),
        'rotate_180': lambda g: np.rot90(g, 2),
        'rotate_270': lambda g: np.rot90(g, 3),
        'invert_colors': lambda g: 10 - g,  # Assuming 10 colors max
        'shift_right': lambda g: np.roll(g, 1, axis=1),
        'shift_down': lambda g: np.roll(g, 1, axis=0),
        'dilate': lambda g: np.kron(g, np.ones((2,2), dtype=int)), # Example structural transformation
        'erode': lambda g: g[::2, ::2], # Example structural transformation
        'border_extract': lambda g: g - np.pad(g[1:-1,1:-1], ((1,1),(1,1)), mode='constant', constant_values=0), # Example structural transformation
        'color_swap': lambda g, mapping: np.vectorize(lambda x: mapping.get(x, x))(g) # Example color mapping
    }

    # Function to apply zero-shot reasoning based on training examples
    # This initiates the pattern detection process.
    def apply_zero_shot_reasoning(task_id, test_input):
        """
        Applies zero-shot reasoning by analyzing training examples to infer transformations.

        This function simulates a reasoning process by examining the provided training
        input-output pairs for a given task ID. It then calls the pattern-first logic
        engine to identify a potential transformation that could explain the examples.

        Args:
            task_id (str): The ID of the current task.
            test_input (list of list of int): The input grid for the test example.

        Returns:
            np.ndarray or None: The predicted output grid as a NumPy array if a pattern
                                is successfully identified and validated, otherwise None.
        """
        if task_id not in train_tasks:
            return None

        train_pairs = train_tasks[task_id]['train']
        # Conceptual prompt building - the logic engine simulates reasoning
        prompt = "Transformations observed in training examples:\n"
        for i, pair in enumerate(train_pairs[:2]):
            prompt += f"Example {i+1}:\n"
            prompt += f"Input:\n{np.array(pair['input'])}\n"
            prompt += f"Output:\n{np.array(pair['output'])}\n\n"
        prompt += f"Apply the same transformation to test input:\n{test_input}\nOutput:"

        # Call the pattern-first logic engine
        return apply_pattern_first_logic(train_pairs, test_input)

    # Core pattern-first logic engine
    # This implements the strategies from Steps 3, 4, and 5.
    def apply_pattern_first_logic(train_pairs, test_input):
        """
        Applies a sequence of pattern-first strategies to predict the output grid.

        This function attempts to solve the task by trying different transformation
        strategies in a specific order: consistent color mappings, geometric transformations
        (flip, rotate), and structural transformations (dilate, erode, border_extract).
        It validates each potential transformation against the training examples and
        selects the one with the best score. If no strategy yields a sufficiently good
        match, it falls back to a backup strategy.

        Args:
            train_pairs (list): A list of dictionaries, each containing 'input' and 'output'
                                grids for the training examples.
            test_input (list of list of int): The input grid for the test example.

        Returns:
            np.ndarray: The predicted output grid as a NumPy array based on the best
                        matching pattern or the backup strategy.
        """
        # Try to detect common patterns based on training examples
        grid_in = np.array(test_input)
        best_score = 0
        best_output = None

        # Check for consistent color mappings (Step 3)
        color_map = detect_color_mapping(train_pairs)
        if color_map:
            candidate = TRANSFORMATIONS['color_swap'](grid_in, color_map)
            score = validate_candidate(train_pairs, candidate)
            if score > best_score:
                best_output = candidate
                best_score = score

        # Try geometric transformations (Step 4)
        for transform_name in ['flip_horizontal', 'flip_vertical', 'rotate_90', 'rotate_180', 'rotate_270']:
            candidate = TRANSFORMATIONS[transform_name](grid_in)
            score = validate_candidate(train_pairs, candidate)
            if score > best_score:
                best_output = candidate
                best_score = score

        # Check structural transformations (Step 5)
        for transform_name in ['dilate', 'erode', 'border_extract']:
            try:
                candidate = TRANSFORMATIONS[transform_name](grid_in)
                score = validate_candidate(train_pairs, candidate)
                if score > best_score:
                    best_output = candidate
                    best_score = score
            except:
                # Ignore errors if transformation is not applicable (e.g., grid too small for erode)
                continue

        # Step 4: If none of the above produce a good match, use a backup strategy (Step 6)
        if best_output is None or validate_candidate(train_pairs, best_output) < 0.7: # Threshold for validation score
             # Use the expected shape from the sample submission for backup strategies
             best_output = apply_backup_strategy(grid_in, task_shapes.get(tid, (len(grid_in), len(grid_in[0]))))

        return best_output

    # Function to detect consistent color mappings from training pairs (Helper for Step 3)
    def detect_color_mapping(train_pairs):
        """
        Detects a consistent color mapping between input and output grids in training pairs.

        Analyzes the training examples to identify if there's a one-to-one mapping
        from input colors to output colors that is consistent across all pairs.

        Args:
            train_pairs (list): A list of dictionaries, each containing 'input' and 'output'
                                grids for the training examples.

        Returns:
            dict or None: A dictionary representing the color mapping (input_color: output_color)
                          if a consistent mapping is found, otherwise None.
        """
        mapping_candidates = {}
        for pair in train_pairs:
            inp = np.array(pair['input'])
            out = np.array(pair['output'])

            if inp.shape != out.shape:
                continue

            unique_pairs = set(zip(inp.flatten(), out.flatten()))
            for in_val, out_val in unique_pairs:
                if in_val not in mapping_candidates:
                    mapping_candidates[in_val] = []
                mapping_candidates[in_val].append(out_val)

        final_map = {}
        for in_val, out_vals in mapping_candidates.items():
            if len(set(out_vals)) == 1:  # Check for consistent mapping
                final_map[in_val] = out_vals[0]

        return final_map if final_map else None

    # Function to validate a candidate output against training examples (Helper for Steps 3, 4, 5)
    def validate_candidate(train_pairs, candidate):
        """
        Validates a candidate output grid against the training examples.

        Compares the candidate output grid with the expected output grids from the
        training pairs, considering rotations and flips for alignment. It calculates
        a score based on the proportion of matching examples or matching cell values.

        Args:
            train_pairs (list): A list of dictionaries, each containing 'input' and 'output'
                                grids for the training examples.
            candidate (np.ndarray): The generated candidate output grid to validate.

        Returns:
            float: A score representing the proportion of matching examples or cell values
                   (between 0.0 and 1.0).
        """
        matches = 0
        total = 0

        for pair in train_pairs:
            inp = np.array(pair['input'])
            out = np.array(pair['output'])

            # Only compare if shapes match
            if candidate.shape != out.shape:
                continue

            # Try to align candidate with training output by trying rotations and flips
            aligned = False
            for _ in range(4):  # Try 4 rotations
                if np.array_equal(candidate, out):
                    matches += 1
                    aligned = True
                    break

                # Check flipped versions
                if np.array_equal(np.flip(candidate, 0), out) or np.array_equal(np.flip(candidate, 1), out):
                    matches += 1
                    aligned = True
                    break

                candidate = np.rot90(candidate)  # Rotate for next iteration

            if not aligned:
                # If no simple alignment works, compare based on cell values (less strict)
                if candidate.size == out.size and candidate.size > 0:
                     matches += np.sum(candidate == out) / candidate.size # Proportion of matching cells
                     aligned = True # Consider it aligned for scoring

            total += 1

        # Return proportion of matching examples/cells as a score
        return matches / total if total > 0 else 0


    # Backup strategies when pattern matching fails (Step 6)
    def apply_backup_strategy(input_grid, expected_shape):
        """
        Applies backup strategies when pattern matching fails to produce a confident prediction.

        This function provides alternative methods for generating an output grid when
        the primary pattern-first logic does not yield a strong match. Strategies include
        pattern repetition (tiling) or a simple color inversion.

        Args:
            input_grid (list of list of int): The input grid for the test example.
            expected_shape (tuple): The expected (rows, columns) shape of the output grid.

        Returns:
            np.ndarray: The generated output grid as a NumPy array using a backup strategy.
        """
        grid = np.array(input_grid)

        # Strategy 1: Pattern repetition (tile the input grid to fill the expected shape)
        if grid.size > 0:
            h, w = expected_shape
            repeated = np.tile(grid, (h//grid.shape[0] + 1, w//grid.shape[1] + 1))
            return repeated[:h, :w]

        # Strategy 2: Color inversion (simple color shift as a last resort)
        return np.vectorize(lambda x: (x + 5) % 10)(grid)

    # Process each test task to generate predictions (Part of Step 7: Resizing and Saving)
    final_predictions = []

    for tid, info in tqdm(test_tasks.items(), desc='Processing tasks'):
        test_input = info['test'][0]['input']
        # Get the expected shape for the output grid (from Step 2)
        expected_shape = task_shapes.get(tid, (len(test_input), len(test_input[0])))

        # Step 1: Apply zero-shot reasoning with pattern-first fallback
        output_grid = apply_zero_shot_reasoning(tid, test_input)

        # Step 2: If reasoning failed, use a backup strategy
        if output_grid is None:
            output_grid = apply_backup_strategy(test_input, expected_shape)

        # Ensure output_grid is not None before accessing shape
        if output_grid is None:
            # Fallback to input grid if all else fails (should not happen with backup strategies)
            output_grid = np.array(test_input)


        # Step 3: Resize the output grid to the expected shape if necessary (Part of Step 7)
        if output_grid.shape != expected_shape:
            output_grid = resize(output_grid, expected_shape, order=0,
                                preserve_range=True, anti_aliasing=False).astype(int)

        # Format the predicted output grid into a string format for submission (Part of Step 7)
        for r in range(output_grid.shape[0]):
            row_str = join_delim.join(map(str, output_grid[r]))
            final_predictions.append({'ID': f'{tid}_{r+1}', 'row': row_str})

    # Save the final predictions to a CSV file (Part of Step 7)
    pd.DataFrame(final_predictions).to_csv(output_csv_path, index=False)
    print(f'Hybrid solution saved to {output_csv_path}')

# Execute the main function to run the solution pipeline
if __name__ == '__main__':
    arc_agi_hybrid_solution(
        train_json_path='train.json',
        test_json_path='test.json',
        sample_csv_path='SampleSubmission.csv',
        output_csv_path='arc_agi_final_submission.csv'
    )

Processing tasks: 100%|██████████| 185/185 [00:00<00:00, 9512.99it/s]

Hybrid solution saved to arc_agi_final_submission.csv


## Visualizing Predictions

This section provides an interactive viewer to compare the test input grids with the predicted output grids generated by the model. This corresponds to the "Visualizing My Predictions" section of the walkthrough.

In [5]:
# Reload extensions for ipywidgets
%reload_ext autoreload
%reload_ext widgetsnbextension

In [6]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings

# Ignore warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Load test data and the generated submission file
with open('/content/test.json', 'r') as f:
    test_data = json.load(f)

submission = pd.read_csv('/content/arc_agi_final_submission.csv')

# Define a colormap for visualizing the grids
cmap = plt.cm.get_cmap('tab20', 10)

# Helper function to extract the predicted grid for a given test task ID from the submission DataFrame
def get_predicted_grid(task_id):
    """
    Retrieves the predicted grid for a given task ID from the submission DataFrame.
    """
    # Filter rows belonging to the current task ID and sort them by ID to maintain order
    group = submission[submission['ID'].str.startswith(task_id + '_')]
    group = group.sort_values(by='ID')
    rows = group['row'].tolist()
    # Convert the string rows back to a numpy array of integers
    return np.array([[int(ch) for ch in row] for row in rows])

# Function to display test input and predicted output grids side-by-side visually and as matrices
def show_grids_side_by_side(input_grid, pred_grid, task_id=''):
    input_grid = np.array(input_grid)
    pred_grid = np.array(pred_grid)

    # === Top Figure: Visual comparison ===
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(f"{task_id} - Test Input vs Predicted Output", fontsize=14, weight='bold')

    ax1 = fig.add_subplot(1, 3, 1)
    ax1.imshow(input_grid, cmap=cmap, vmin=0, vmax=9)
    ax1.set_title("📥 Test Input")
    ax1.axis('off')

    ax2 = fig.add_subplot(1, 3, 2)
    ax2.imshow(pred_grid, cmap=cmap, vmin=0, vmax=9)
    ax2.set_title("📤 Predicted Output")
    ax2.axis('off')

    ax3 = fig.add_subplot(1, 3, 3)
    legend = np.arange(10).reshape(-1, 1)
    ax3.imshow(legend, cmap=cmap, vmin=0, vmax=9)
    ax3.set_title("🎨 Color Key", fontsize=10)
    ax3.set_yticks(range(10))
    ax3.set_yticklabels(range(10))
    ax3.set_xticks([])
    ax3.tick_params(axis='y', labelsize=10)
    ax3.grid(False)

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()

    # === Below: Always-visible raw matrices side-by-side ===
    input_box = widgets.Output()
    pred_box = widgets.Output()

    with input_box:
        print("📘 Test Input Grid:")
        print(input_grid)

    with pred_box:
        print("🟥 Model Predicted Output:")
        print(pred_grid)

    # Display the matrix representations side-by-side
    display(widgets.HBox([input_box, pred_box]))

# Function to display a specific test task and its prediction using the viewer components
def display_test_task(task_ids, index):
    task_id = task_ids[index]
    print(f"\n🔍 Viewing Test Task: {task_id} ({index+1}/{len(test_ids)})")

    test_input = test_data[task_id]['test'][0]['input']

    try:
        # Get the predicted grid and display it using the side-by-side function
        test_pred = get_predicted_grid(task_id)
        show_grids_side_by_side(test_input, test_pred, task_id=task_id)
    except Exception as e:
        # Handle cases where a prediction might not be found or an error occurs during retrieval/display
        print(f"⚠️ Could not display prediction for this task: {e}")
        # Optionally display the input grid even if prediction is missing, with a placeholder for prediction
        show_grids_side_by_side(test_input, np.zeros_like(test_input) if test_input else np.array([[0]]), task_id=task_id + " (Prediction Missing)")


# Setup navigation for iterating through test tasks
test_ids = list(test_data.keys())
test_index = 0

# Navigation buttons with styling
prev_button = widgets.Button(description="⏮️ Previous", button_style='warning')
next_button = widgets.Button(description="▶️ Next", button_style='success')
jump_box = widgets.BoundedIntText(value=1, min=1, max=len(test_ids), description="Jump to:")
jump_button = widgets.Button(description="Go", button_style='primary')
output_area = widgets.Output() # Output area to display the test task viewer

# Navigation button handlers to update the displayed test task
def render_current_task():
    with output_area:
        clear_output(wait=True)
        display_test_task(test_ids, test_index)

def on_prev_clicked(b):
    global test_index
    test_index = (test_index - 1) % len(test_ids)
    render_current_task()

def on_next_clicked(b):
    global test_index
    test_index = (test_index + 1) % len(test_ids)
    render_current_task()

def on_jump_clicked(b):
    global test_index
    new_index = jump_box.value - 1
    if 0 <= new_index < len(test_ids):
        test_index = new_index
        render_current_task()

# Link button clicks to their respective handler functions
prev_button.on_click(on_prev_clicked)
next_button.on_click(on_next_clicked)
jump_button.on_click(on_jump_clicked)

# Display the title and navigation UI for the test task viewer
display(HTML("""
    <div style='width: 100%; text-align: 20%; padding-bottom: 10px;'>
        <h2 style='color:#1f77b4; font-weight:bold; font-size:24px;'>
            🛰️ ARC Viewer: Test Inputs and My Predictions
        </h2>
    </div>
"""))

nav_ui = widgets.HBox([prev_button, next_button, jump_box, jump_button])
display(nav_ui, output_area)

# Initial render of the first test task
render_current_task()

Output()